# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citiesDF = pd.read_csv("../output_data/cities.csv")
citiesDF.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure Locations and Weights
locations = citiesDF[['Lat', 'Lng']]
weight = citiesDF['Humidity'].astype(float)

# Configure gmap
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure()

# add a marker latyer on heatmap
marker_layer = gmaps.marker_layer(locations)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
citiesDF

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [5]:
# The perfect weather conditions! (Drop all rows that doesnt satisfy conditions):
# Humidity < 60
# Temp: < 85 and > 70
# Cloudiness < 40

citiesDF.drop(citiesDF[citiesDF["Humidity"] > 60].index, inplace= True)
citiesDF.drop(citiesDF[(citiesDF["Max Temp"] < 70) | (citiesDF["Max Temp"] > 85)].index, inplace= True)
citiesDF.drop(citiesDF[citiesDF["Cloudiness"] > 40].index, inplace= True)
citiesDF

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
15,15,albany,40,US,1558378727,45,42.65,-73.75,82.40,12.75
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
75,75,saint george,40,GR,1558378768,33,39.45,22.34,75.20,11.41
144,144,plainview,40,US,1558378590,37,34.18,-101.71,84.99,18.34
153,153,porto novo,20,BR,1558378783,53,-23.68,-45.44,77.00,4.70
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
from pprint import pprint

In [7]:
# Store citiesDF into hotel_df and add "Hotel Name" column to DataFrame
hotel_df = citiesDF


baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#search for the hotels

target_radius = 5000
target_type = "hotel"


# dictionary of parameters to update at each iteration 
parameters = {
    "type": target_type,
    "radius" :target_radius,
    "key": g_key
}

# use iterrows() to iterate through the rows in the DF
for index, row in hotel_df.iterrows():
    #grab the latitudes and longitudes from the columns in the DF
    latitude = row['Lat']
    longitude = row['Lng']
    
    # change location each iteration while leaving original params in place
    parameters["location"] = f"{latitude},{longitude}"
    
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_response = requests.get(base_url, params=parameters)
  

    # convert to json
    hotel_json = hotel_response.json()
    #pprint(hotel_json) # to check the indices of the hotel name 
    
 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][1]["name"]
        print(f"First hotel in {target_radius} meters is {hotel_json['results'][1]['name']}")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("--------------------------------------")

First hotel in 5000 meters is PEP Opuwo
--------------------------------------
First hotel in 5000 meters is Hotel Posada San Agustin
--------------------------------------
First hotel in 5000 meters is Hilton Albany
--------------------------------------
First hotel in 5000 meters is Hotel San Isidro Oficial
--------------------------------------
First hotel in 5000 meters is Pousada Convento Beja
--------------------------------------
First hotel in 5000 meters is Hotel Escarpas do Lago Capitólio
--------------------------------------
First hotel in 5000 meters is Kotsiras Christos
--------------------------------------
First hotel in 5000 meters is Wayland Baptist University
--------------------------------------
First hotel in 5000 meters is Harmonia
--------------------------------------
First hotel in 5000 meters is Hôtel La Giralda
--------------------------------------
First hotel in 5000 meters is DETAILS Vale da Lapa Village Resort
--------------------------------------
First

In [8]:
# checking the DF with the hotel names in the new column
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,PEP Opuwo
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11,Hotel Posada San Agustin
15,15,albany,40,US,1558378727,45,42.65,-73.75,82.40,12.75,Hilton Albany
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Hotel San Isidro Oficial
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,Pousada Convento Beja


In [9]:
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))